# 2 Selecting a Column or Subset of Columns (选中一列，或列的子集)

如果一个GroupBy对象是由DataFrame创建来的，那么通过列名或一个包含列名的数组来对GroupBy对象进行索引的话，就相当于对列取子集做聚合（column subsetting for aggregation）。这句话的意思是：

```
df.groupby('key1')['data1'] 
df.groupby('key1')[['data2']]
```
上面的代码其实就是下面的语法糖（Syntactic sugar）：
```
df['data1'].groupby(df['key1']) 
df[['data2']].groupby(df['key1'])
```
> 语法糖(Syntactic sugar),是由Peter J. Landin(和图灵一样的天才人物，是他最先发现了Lambda演算，由此而创立了函数式编程)创造的一个词语，它意指那些没有给计算机语言添加新功能，而只是对人类来说更“甜蜜”的语法。语法糖往往给程序员提供了更实用的编码方式，有益于更好的编码风格，更易读。不过其并没有给语言添加什么新东西。

尤其是对于一些很大的数据集，这种用法可以聚集一部分列。例如，在处理一个数据集的时候，想要只计算data2列的平均值，并将结果返还为一个DataFrame，我们可以这样写：


In [2]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'], 
                   'data1' : np.random.randn(5), 
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.455376,-2.202801
1,a,two,-0.749594,0.120302
2,b,one,-0.533130,0.450771
3,b,two,0.053130,0.615951
4,a,one,0.143152,-0.479462


In [3]:
df.groupby(['key1','key2'])['data2']

In [4]:
df.groupby(['key1','key2'])[['data2']]

###  注意这两种格式生成的对象是不一样的(一个是series，一个是DataFrame)

In [5]:
df.groupby(['key1','key2'])['data2'].mean()

key1  key2
a     one    -1.341131
      two     0.120302
b     one     0.450771
      two     0.615951
Name: data2, dtype: float64

In [6]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one  -1.341131
     two   0.120302
b    one   0.450771
     two   0.615951

## 利用字典的形式进行分组

In [7]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,0.333382,-0.921473,-0.727373,-2.481996,-1.255048
Steve,-0.181599,-1.209607,-0.510325,-0.348148,-1.098583
Wes,1.942423,-1.156609,-1.173209,0.327922,0.374301
Jim,-0.579911,1.363422,0.426088,-1.021044,1.184105
Travis,0.627946,-2.258573,0.794519,-0.588704,0.583411


## 自定义分组的规则，利用字典

In [8]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f': 'orange'}
# 这里的字典是没有“f”的，这对于要分组的DF是没有影响的,
people.groupby(mapping,axis = 1).sum()

,blue,red
Joe,-3.209369,-1.843139
Steve,-0.858473,-2.489789
Wes,-0.845287,1.160116
Jim,-0.594955,1.967616
Travis,0.205815,-1.047216


## 对多列进行和多函数进行不同聚合函数函数的应用
```
grouped_pct.agg(['mean', 'std', peak_to_peak])
```
> 这里mean，和std的函数是自定义的韩束，而peak_to_peak 函数是自定义额度函数，我们可以看见都可以利用agg()函数进行统一使用

上面我们把多个聚合函数作为一个list传入给agg，这些函数会独立对每一个组进行计算。

上面结果的列名是自动给出的，当然，我们也可以更改这些列名。这种情况下，传入一个由tuple组成的list，每个tuple的格式是`(name, function)`，每个元组的第一个元素会被用于作为DataFrame的列名（我们可以认为这个二元元组list是一个有序的映射）：
```
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])
```

## 不使用多层级索引，这个很有必要
> 在使用groupby函数中调用方法as_index = False，这个方法可以是我在分组聚合之后不在使用reset_index()的方法